## **Lab01: MỐI QUAN HỆ CỦA DỮ LIỆU**

**Giáo viên hướng dẫn:** 
- Lê Ngọc Thành

---

## **Import các thư viện**

In [12]:
import pandas as pd
import requests
from datetime import datetime, timedelta, date
from bs4 import BeautifulSoup
import requests
import csv
import datetime

## **Crawl Data từ trang web**

In [13]:
def detect_text(row, tag):
    element = BeautifulSoup(row, 'html.parser').find_all(tag)
    text = [col.get_text() for col in element]
    return text

In [14]:
html = requests.get('https://www.worldometers.info/coronavirus/').text
html_soup = BeautifulSoup(html, 'html.parser')

table=html_soup.find_all("table", {"id": "main_table_countries_yesterday"})
rows=table[0].find_all('tr')
table_new = rows[8:237]

heading = rows[0]
col_name = detect_text(str(heading), 'th')[0:16]

In [15]:
dt = datetime.datetime.today()


with open(f'RawData/Covid19-raw-{dt.day-1}-{dt.month}.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f,delimiter=',')
    for i in range (len(col_name)):
                if '\n' in str(col_name[i]):
                    col_name[i]=str(col_name[i]).replace('\n','')
                if "Cases/1M pop" in  str(col_name[i]):
                    col_name[i]="TotalCases/1M pop"
    writer.writerow(col_name)

    for row in table_new:
        test_data = detect_text(str(row), 'td')[0:16]
        for i in range (len(test_data)):
            if '\n' in str(test_data[i]):
                test_data[i]=str(test_data[i]).replace('\n','')
        writer.writerow(test_data)

## **Tiền xử lý**

- Đọc dữ liệu lên từ Folder RawData

In [29]:
dt = datetime.datetime.today()

covid_df = pd.read_csv(f'RawData/Covid19-raw-{dt.day-1}-{dt.month}.csv')
covid_df

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent
0,NaN,World,"516,494,463","+513,452","6,274,577","+1,999","471,161,199","+590,963","39,058,687","40,263","66,261",805.0,NaN,NaN,NaN,All
1,1.0,China,"218,945",+374,"5,153",+12,"202,277","+1,602","11,515",659,152,4,"160,000,000","111,163","1,439,323,776",Asia
2,2.0,USA,"83,534,060","+77,116","1,024,386",+291,"80,910,509","+38,417","1,599,165","1,724","249,672","3,062","1,011,100,248","3,022,041","334,575,277",North America
3,3.0,India,"43,098,743","+4,195","524,024",+22,"42,554,416","+6,717","20,303",698,"30,676",373,"839,844,925","597,764","1,404,976,711",Asia
4,4.0,Brazil,"30,543,908","+19,725","664,143",+176,"29,612,822","+3,728","266,943","8,318","141,840","3,084","63,776,166","296,164","215,340,527",South America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,224.0,MS Zaandam,9,NaN,2,NaN,7,NaN,0,NaN,NaN,NaN,NaN,NaN,,NaN
225,225.0,Niue,9,NaN,,NaN,8,NaN,1,NaN,"5,468",NaN,NaN,NaN,"1,646",Australia/Oceania
226,226.0,Micronesia,7,NaN,,NaN,1,NaN,6,NaN,60,NaN,NaN,NaN,"117,242",Australia/Oceania
227,227.0,Nauru,5,NaN,,NaN,3,NaN,2,NaN,457,NaN,NaN,NaN,"10,949",Australia/Oceania


- Xóa các dấu `"+", ","` và thay các ô `"N/A"` thành `0`
- Thay các dòng ở cột `Continent` bị trống thành `"The remaining areas"`

In [30]:
cols = covid_df.columns
row = covid_df.shape[0]
for col in cols:
    for i in range(row):
        if "," in str(covid_df[col][i]) or "+" in str(covid_df[col][i]) or "N/A" in str(covid_df[col][i]) :
            covid_df[col][i]=str(covid_df[col][i]).replace(',','')
            covid_df[col][i]=str(covid_df[col][i]).replace('+','')
        if pd.isna(covid_df[col][i])==True or covid_df[col][i]==" "or covid_df[col][i]=="":
            covid_df[col][i]=0
for i in range(row):
    if covid_df["Continent"][i] ==0:
        covid_df["Continent"][i] = "The remaining areas"

C:\Users\123\AppData\Local\Temp/ipykernel_16452/876558283.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df[col][i]=0
C:\Users\123\AppData\Local\Temp/ipykernel_16452/876558283.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df[col][i]=str(covid_df[col][i]).replace(',','')
C:\Users\123\AppData\Local\Temp/ipykernel_16452/876558283.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df[col][i]=str(covid_df[col][

- Các cột dữ liệu  không phải `Country,Other` và `Continent` sẽ được chuyển về **numeric**

In [31]:
for col in cols:
    if col not in ["Country,Other","Continent"]:
        covid_df[col] = pd.to_numeric(covid_df[col])

In [32]:
covid_df.dtypes

#                    float64
Country,Other         object
TotalCases             int64
NewCases               int64
TotalDeaths            int64
NewDeaths              int64
TotalRecovered         int64
NewRecovered           int64
ActiveCases            int64
Serious,Critical       int64
TotalCases/1M pop      int64
Deaths/1M pop        float64
TotalTests             int64
Tests/1M pop           int64
Population             int64
Continent             object
dtype: object

- Có vẻ kiểu dữ liệu của các cột đã hợp lý.

- Với dòng World ta có thể thấy bị thiếu dữ liệu ở 3 cột `Total Tests, Tests/1M pop,Population`. Vì vậy ta sẽ tính luôn cho dòng này.

In [33]:
covid_df["TotalTests"][0] = covid_df["TotalTests"].sum()
covid_df["Tests/1M pop"][0] = covid_df["Tests/1M pop"].sum()
covid_df["Population"][0] = covid_df["Population"].sum()

C:\Users\123\AppData\Local\Temp/ipykernel_16452/115733474.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df["TotalTests"][0] = covid_df["TotalTests"].sum()
C:\Users\123\AppData\Local\Temp/ipykernel_16452/115733474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df["Tests/1M pop"][0] = covid_df["Tests/1M pop"].sum()
C:\Users\123\AppData\Local\Temp/ipykernel_16452/115733474.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

- Ghi dữ liệu vào folder **ProcessedData** với định dạng là ``Covid19-{day}-{month}.csv``

In [34]:
dt = datetime.datetime.today()

covid_df.to_csv(f'ProcessedData/Covid19-{dt.day-1}-{dt.month}.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=False, index_label=False, mode='w', encoding=None, compression='infer', quoting=None, quotechar='"', line_terminator=None, chunksize=None, date_format=None, doublequote=True, escapechar=None, decimal='.', errors='strict', storage_options=None)

In [35]:
covid_df

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotalCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent
0,0.0,World,516494463,513452,6274577,1999,471161199,590963,39058687,40263,66261,805.0,6302745461,409517542,7898240156,All
1,1.0,China,218945,374,5153,12,202277,1602,11515,659,152,4.0,160000000,111163,1439323776,Asia
2,2.0,USA,83534060,77116,1024386,291,80910509,38417,1599165,1724,249672,3062.0,1011100248,3022041,334575277,North America
3,3.0,India,43098743,4195,524024,22,42554416,6717,20303,698,30676,373.0,839844925,597764,1404976711,Asia
4,4.0,Brazil,30543908,19725,664143,176,29612822,3728,266943,8318,141840,3084.0,63776166,296164,215340527,South America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,224.0,MS Zaandam,9,0,2,0,7,0,0,0,0,0.0,0,0,0,The remaining areas
225,225.0,Niue,9,0,0,0,8,0,1,0,5468,0.0,0,0,1646,Australia/Oceania
226,226.0,Micronesia,7,0,0,0,1,0,6,0,60,0.0,0,0,117242,Australia/Oceania
227,227.0,Nauru,5,0,0,0,3,0,2,0,457,0.0,0,0,10949,Australia/Oceania


## **Trực quan mối quan hệ giữa các trường dữ liệu**